In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from negmas.helpers.inout import load
from negmas.preferences.ops import opposition_level
from negmas.inout import Scenario
from pathlib import Path
from math import sqrt
import warnings
from utils import DTYPESFINAL, read_data, refresh_values, TAUNATIVE
from pandas.errors import DtypeWarning, SettingWithCopyWarning
from rich import print
warnings.filterwarnings("ignore", category=DtypeWarning)
warnings.filterwarnings("ignore", category=SettingWithCopyWarning)
warnings.filterwarnings("ignore", category=UserWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 3)
#sns.set_context("talk", rc={"legend.fontsize": 20})
sns.set_context("poster")
MOVE_LEGEND_UP = True
sns.set_style("darkgrid")
from itertools import product

In [3]:
files =  [f"serverclean/y{i}/y{i}.csv" for i in range(2010, 2019) if i != 2014] + ["serverclean/final"]
strategies, real, found, expected, domains = dict(), dict(), dict(), dict(), dict()
for filename in files:
    data , agents = read_data([Path.cwd().parent.parent / filename], implicit_failures=False, failures=False)
    data = data.loc[(~data.Strategy.str.contains("CAB")) & (~data.Strategy.str.contains("WAR")), :]
    for c in ["Strategy", "Strategy1", "Strategy2"]:
        data[c] = data[c].astype(str)
    data.loc[data.Strategy1 == data.Strategy2, "Strategy"] = data.loc[data.Strategy1 == data.Strategy2, "Strategy1"]
    for c in ["Strategy", "Strategy1", "Strategy2"]:
        data[c] = data[c].astype("category")
    agents = agents.loc[(~agents.Strategy.str.contains("CAB")) & (~agents.Strategy.str.contains("WAR")) &
                        (~agents.StrategyPartner.str.contains("CAB")) & (~agents.StrategyPartner.str.contains("WAR"))
                        , :]
    
    s =  set(data.Strategy1.unique()).union(data.Strategy2.unique())
    n = len(s)
    strategies[filename] = n * n
    found[filename] = set(data.Strategy.astype(str).unique())
    real[filename] = len(found[filename])
    expected[filename] = set(f"{a}-{b}" if a != b else a for a, b in product(s, s))
    domains[filename] = data.Domain.unique()

Found 206 total records

Will use 206 data points

Found 1963 total records

Will use 1963 data points

Found 3386 total records

...

Found 3727 total records

Will use 3727 data points

Adding final/final.csv

Found 280091 total records

Will use 280091 data points

In [4]:
print(strategies, real)

{
    'serverclean/y2010/y2010.csv': 16,
    'serverclean/y2011/y2011.csv': 64,
    'serverclean/y2012/y2012.csv': 64,
    'serverclean/y2013/y2013.csv': 49,
    'serverclean/y2015/y2015.csv': 529,
    'serverclean/y2016/y2016.csv': 256,
    'serverclean/y2017/y2017.csv': 324,
    'serverclean/y2018/y2018.csv': 400,
    'serverclean/final': 400
}
{
    'serverclean/y2010/y2010.csv': 16,
    'serverclean/y2011/y2011.csv': 64,
    'serverclean/y2012/y2012.csv': 64,
    'serverclean/y2013/y2013.csv': 49,
    'serverclean/y2015/y2015.csv': 529,
    'serverclean/y2016/y2016.csv': 247,
    'serverclean/y2017/y2017.csv': 319,
    'serverclean/y2018/y2018.csv': 400,
    'serverclean/final': 127
}

In [5]:
print(sum(strategies.values()), sum(real.values()), sum(real.values()) / sum(strategies.values()))

2102 1815 0.8634633682207421

In [6]:
from math import sqrt
print(sum([sqrt(_) for _ in strategies.values()]))
print(sum([sqrt(_) for _ in real.values()]))

124.0

114.84623241457811

In [7]:
for k, e in expected.items():
    f = found.get(k, set())
    extra = f.difference(e)
    if extra:
        print(f"{k} Extra: {extra}")
    missing = e.difference(f)
    if missing:
        print(f"{k} Missing: {missing}")

serverclean/y2016/y2016.csv Missing: {'Ngent-ClockworkAgent', 'Ngent-MaxOops', 'Ngent-AgentSmith2016', 
'Ngent-SYAgent', 'ClockworkAgent-Ngent', 'AgentSmith2016-Ngent', 'AgentLight-Ngent', 'SYAgent-Ngent', 
'Ngent-AgentLight'}

serverclean/y2017/y2017.csv Missing: {'Mosa-PonPokoAgent', 'Rubick-Gin', 'Imitator-Rubick', 'Rubick-Imitator', 
'Gin-Rubick'}

serverclean/final Missing: {'Boulware-Mamenchis', 'Libra-TucAgent', 'Caduceus-Mosa', 'Yushu-AgentK', 
'Yeela-Linear', 'HardHeaded-Mosa', 'CUHK-IAMhaggler', 'Yushu-IAMhaggler', 'Yushu-Caduceus', 'IAMhaggler-Mamenchis',
'Group3-Linear', 'IAMhaggler-Atlas3', 'IAMhaggler-CUHK', 'HardHeaded-Yeela', 'Yushu-Mosa', 'HardHeaded-TucAgent', 
'Conceder-IAMhaggler', 'Nozomi-AgentK', 'Mamenchis-Mosa', 'HardHeaded-Nozomi', 'Group3-Yeela', 'AgentK-Mosa', 
'AgentK-Group3', 'AgentK-IAMhaggler', 'Mosa-AgentK', 'AgentGG-IAMhaggler', 'Mamenchis', 'Mamenchis-Boulware', 
'Boulware-Libra', 'Nozomi-NiceTfT', 'Linear-Libra', 'Yushu-Mamenchis', 'Group3-AgentGG', 'Mosa-CUHK', 
'Conceder-Group3', 'Boulware-IAMhaggler', 'Mosa-Linear', 'Group3-Atlas3', 'Libra-Atlas3', 'NiceTfT-TucAgent', 
'Yeela-IAMhaggler', 'HardHeaded-IAMhaggler', 'NiceTfT-Group3', 'Libra-NiceTfT', 'HardHeaded-Yushu', 'AgentGG-Mosa',
'Nozomi-AgentGG', 'Yeela-Caduceus', 'Group3-AgentK', 'Mamenchis-Yeela', 'Atlas3-Mamenchis', 'Caduceus-Libra', 
'Caduceus-Yushu', 'Mosa-Yushu', 'Mosa-Conceder', 'IAMhaggler-Yeela', 'Yeela-NiceTfT', 'Mamenchis-Atlas3', 
'Boulware-Nozomi', 'Mosa-MiCRO', 'Caduceus-Group3', 'CUHK-Yeela', 'TucAgent-Linear', 'Mosa-NiceTfT', 
'Boulware-Group3', 'MiCRO-TucAgent', 'Conceder-Libra', 'Nozomi-Boulware', 'Libra-AgentGG', 'Linear-Mosa', 
'Mosa-Yeela', 'Yeela-AgentGG', 'Libra-Yushu', 'IAMhaggler-Boulware', 'Boulware-Yushu', 'Libra-Boulware', 
'Atlas3-Nozomi', 'TucAgent-AgentK', 'Yushu-Atlas3', 'Group3-TucAgent', 'Libra-Linear', 'CUHK-TucAgent', 
'AgentK-TucAgent', 'MiCRO-IAMhaggler', 'Mosa', 'MiCRO-Mamenchis', 'Yeela-Mosa', 'Libra-Yeela', 'AgentGG-Nozomi', 
'Atlas3-Mosa', 'Yushu-Boulware', 'Linear-Yeela', 'Mosa-Group3', 'Nozomi-CUHK', 'IAMhaggler-AgentGG', 
'Yeela-Nozomi', 'Mamenchis-Linear', 'Yushu-CUHK', 'Mamenchis-Conceder', 'Libra-CUHK', 'Mosa-IAMhaggler', 
'Libra-Conceder', 'TucAgent-Mamenchis', 'Mamenchis-Caduceus', 'TucAgent-HardHeaded', 'Conceder-Nozomi', 
'Libra-IAMhaggler', 'Conceder-Mamenchis', 'TucAgent-Mosa', 'Mosa-HardHeaded', 'Libra-Caduceus', 'NiceTfT-Yushu', 
'CUHK-Libra', 'TucAgent-Atlas3', 'Yeela-Atlas3', 'IAMhaggler-Mosa', 'Group3-Libra', 'Mosa-Atlas3', 'Group3-MiCRO', 
'Mosa-Mamenchis', 'IAMhaggler-HardHeaded', 'Caduceus-IAMhaggler', 'Yushu-Linear', 'Boulware-TucAgent', 
'Mamenchis-Yushu', 'Nozomi-Group3', 'Yeela-Boulware', 'Group3-CUHK', 'AgentGG-Mamenchis', 'CUHK-Mosa', 'Group3', 
'TucAgent-Boulware', 'NiceTfT-Mamenchis', 'Mosa-Libra', 'Yushu-MiCRO', 'Nozomi-HardHeaded', 'Mamenchis-AgentGG', 
'NiceTfT-Mosa', 'Libra-Mosa', 'IAMhaggler-Conceder', 'Yeela-Mamenchis', 'Mosa-Caduceus', 'TucAgent-Libra', 
'AgentGG-TucAgent', 'MiCRO-Yushu', 'TucAgent-AgentGG', 'Libra-HardHeaded', 'MiCRO-Group3', 'TucAgent-Yushu', 
'Yushu-Conceder', 'Nozomi-TucAgent', 'Conceder-Mosa', 'Atlas3-Libra', 'Mamenchis-AgentK', 'Group3-Nozomi', 
'NiceTfT-Libra', 'HardHeaded-Libra', 'Group3-Caduceus', 'Group3-HardHeaded', 'Nozomi-Yeela', 'Yeela-AgentK', 
'Linear-Group3', 'IAMhaggler-AgentK', 'Atlas3-IAMhaggler', 'Atlas3-Yushu', 'Linear-Yushu', 'Caduceus-Nozomi', 
'Mamenchis-CUHK', 'Caduceus-TucAgent', 'TucAgent-Caduceus', 'Nozomi-Linear', 'TucAgent-NiceTfT', 'Atlas3-Yeela', 
'Libra', 'Yushu-TucAgent', 'IAMhaggler-Group3', 'Mosa-Boulware', 'Atlas3-Group3', 'Mamenchis-MiCRO', 
'Linear-Nozomi', 'Group3-Conceder', 'Yushu-HardHeaded', 'Group3-NiceTfT', 'Libra-Group3', 'TucAgent-CUHK', 
'TucAgent-Yeela', 'Yushu-Yeela', 'CUHK-Group3', 'Yeela', 'IAMhaggler-TucAgent', 'NiceTfT-Nozomi', 'Yushu-Libra', 
'Caduceus-Mamenchis', 'Yushu-AgentGG', 'IAMhaggler-MiCRO', 'IAMhaggler-Yushu', 'Group3-Mosa', 'AgentGG-Yushu', 
'Mosa-Nozomi', 'HardHeaded-Mamenchis', 'TucAgent-Nozomi', 'Mamenchis-Libra', 'Group3-Yushu', 'Nozomi-Atlas3', 
'Group3-IAMhaggler', 'Nozomi-Caduceus', 'AgentK-Mamenchis', 'Caduceus-Yeela', 'Mamenchis-Nozomi', 'Libra-AgentK', 
'Nozomi-MiCRO', 'Linear-TucAgent', 'Linear-IAMhaggler', 'Yushu-Nozomi', 'TucAgent-Conceder', 'Nozomi-Mamenchis', 
'Nozomi-Yushu', 'Nozomi-Libra', 'Libra-Nozomi', 'Ag

In [8]:
print(domains)

{
    'serverclean/y2010/y2010.csv': ['ItexvsCypress', 'EnglandZimbabwe', 'Travel']
Categories (3, object): ['EnglandZimbabwe', 'ItexvsCypress', 'Travel'],
    'serverclean/y2011/y2011.csv': ['Car', 'Camera', 'NiceOrDie', 'Laptop', 'Acquisition', 'Grocery', 'Amsterdam', 
'Energy']
Categories (8, object): ['Acquisition', 'Amsterdam', 'Camera', 'Car', 'Energy', 'Grocery', 'Laptop', 'NiceOrDie'],
    'serverclean/y2012/y2012.csv': ['EnglandvsZimbabwe', 'Phone', 'EnergySmall', 'Acquisition', 'Outfit', ..., 
'Fitness', 'FlightBooking', 'Barbecue', 'HouseKeeping', 'MusicCollection']
Length: 15
Categories (15, object): ['Acquisition', 'AirportSiteSelection', 'Barbecue', 'Barter', ..., 'Outfit', 'Phone', 
'RentalHouse', 'Supermarket'],
    'serverclean/y2013/y2013.csv': ['Kitchen', 'Fifty2013', 'planes', 'Coffee', 'AcquisitionA', ..., 'Icecream', 
'Ultimatum', 'DogChoosing', 'cameradomain', 'Wholesaler']
Length: 14
Categories (14, object): ['AcquisitionA', 'Animal', 'Coffee', 'DefensiveCharms', ..., 'Ultimatum', 'Wholesaler', 
'cameradomain', 'planes'],
    'serverclean/y2015/y2015.csv': ['KillerRobot', 'Symposium', 'Politics', 'Movie', 'BankRobbery', ..., 'Tram', 
'Holiday', 'Dinner', 'Vacation', 'University']
Length: 15
Categories (15, object): ['BankRobbery', 'BuildingConstruction', 'CarDomain', 'CarPurchase', ..., 'Tram', 
'University', 'Vacation', 'ZoningPlan'],
    'serverclean/y2016/y2016.csv': ['ElectricVehicle', 'TriangularFight', 'TwF', 'SmartGrid', 'SmartGridDomain', 
..., 'KDomain', 'SmartEnergyGridLarge', 'NewDomain', 'WindFarm', 'EnergyLarge']
Length: 15
Categories (15, object): ['DomainAce', 'ElectricVehicle', 'EnergyLarge', 'JapanTrip2016', ..., 'SmartGridDomain', 
'TriangularFight', 'TwF', 'WindFarm'],
    'serverclean/y2017/y2017.csv': ['LunchTime', 'SmartGrid', 'MovieTime', 'MusicalInstuments', 'Tangxun', 'Music',
'Supermarket', 'AlphabetMaze', 'GeneJack', 'MyDomain']
Categories (10, object): ['AlphabetMaze', 'GeneJack', 'LunchTime', 'MovieTime', ..., 'MyDomain', 'SmartGrid', 
'Supermarket', 'Tangxun'],
    'serverclean/y2018/y2018.csv': ['IQSon', 'BetaOne', 'Hamada', 'MengWan']
Categories (4, object): ['BetaOne', 'Hamada', 'IQSon', 'MengWan'],
    'serverclean/final': ['AlphabetMaze', 'Music', 'BetaOne', 'LunchTime', 'MusicalInstuments', ..., 
'domain33@2021', 'domain22@2021', 'domain07@2021', 'domain15@2021', 'domain28@2021']
Length: 171
Categories (184, object): ['AlphabetMaze', 'BetaOne', 'BuildingConstruction', 'CarDomain', ..., 'domain41@2021', 
'domain45@2021', 'domain47@2021', 'domain49@2021']
}

In [9]:
print({k: len(v) for k, v in domains.items()})
print(sum(len(v) for k, v in domains.items()))

{
    'serverclean/y2010/y2010.csv': 3,
    'serverclean/y2011/y2011.csv': 8,
    'serverclean/y2012/y2012.csv': 15,
    'serverclean/y2013/y2013.csv': 14,
    'serverclean/y2015/y2015.csv': 15,
    'serverclean/y2016/y2016.csv': 15,
    'serverclean/y2017/y2017.csv': 10,
    'serverclean/y2018/y2018.csv': 4,
    'serverclean/final': 171
}

255

In [10]:
print(found)

{
    'serverclean/y2010/y2010.csv': {
        'Yushu-AgentK',
        'Yushu',
        'IAMhaggler-Nozomi',
        'Nozomi',
        'AgentK-Yushu',
        'Nozomi-IAMhaggler',
        'Yushu-Nozomi',
        'IAMhaggler-AgentK',
        'IAMhaggler',
        'Nozomi-AgentK',
        'AgentK',
        'Nozomi-Yushu',
        'AgentK-IAMhaggler',
        'Yushu-IAMhaggler',
        'AgentK-Nozomi',
        'IAMhaggler-Yushu'
    },
    'serverclean/y2011/y2011.csv': {
        'AgentK2-Gahboninho',
        'AgentK2-ValueModelAgent',
        'NiceTfT-BramAgent',
        'ValueModelAgent-NiceTfT',
        'BramAgent-HardHeaded',
        'NiceTfT-TheNegotiator',
        'HardHeaded-ValueModelAgent',
        'HardHeaded-BramAgent',
        'NiceTfT-HardHeaded',
        'AgentK2-TheNegotiator',
        'IAMhaggler2011-Gahboninho',
        'NiceTfT-ValueModelAgent',
        'AgentK2-BramAgent',
        'TheNegotiator-AgentK2',
        'HardHeaded-IAMhaggler2011',
        'HardHeaded-TheNegotiator',
        'BramAgent-TheNegotiator',
        'Gahboninho-AgentK2',
        'AgentK2',
        'NiceTfT-Gahboninho',
        'TheNegotiator-ValueModelAgent',
        'AgentK2-HardHeaded',
        'BramAgent-Gahboninho',
        'IAMhaggler2011-BramAgent',
        'ValueModelAgent-BramAgent',
        'HardHeaded-AgentK2',
        'Gahboninho',
        'IAMhaggler2011-AgentK2',
        'Gahboninho-HardHeaded',
        'TheNegotiator-NiceTfT',
        'ValueModelAgent-Gahboninho',
        'Gahboninho-ValueModelAgent',
        'IAMhaggler2011-TheNegotiator',
        'Gahboninho-BramAgent',
        'HardHeaded-NiceTfT',
        'BramAgent-ValueModelAgent',
        'ValueModelAgent',
        'BramAgent-AgentK2',
        'TheNegotiator-Gahboninho',
        'Gahboninho-TheNegotiator',
        'HardHeaded',
        'IAMhaggler2011-ValueModelAgent',
        'IAMhaggler2011-NiceTfT',
        'ValueModelAgent-IAMhaggler2011',
        'AgentK2-NiceTfT',
        'Gahboninho-IAMhaggler2011',
        'ValueModelAgent-TheNegotiator',
        'Gahboninho-NiceTfT',
        'IAMhaggler2011',
        'IAMhaggler2011-HardHeaded',
        'HardHeaded-Gahboninho',
        'TheNegotiator-BramAgent',
        'ValueModelAgent-HardHeaded',
        'TheNegotiator',
        'BramAgent',
        'BramAgent-NiceTfT',
        'TheNegotiator-IAMhaggler2011',
        'NiceTfT-AgentK2',
        'BramAgent-IAMhaggler2011',
        'ValueModelAgent-AgentK2',
        'NiceTfT',
        'NiceTfT-IAMhaggler2011',
        'TheNegotiator-HardHeaded',
        'AgentK2-IAMhaggler2011'
    },
    'serverclean/y2012/y2012.csv': {
        'AgentMR-TheNegotiatorReloaded',
        'AgentMR-OMACagent',
        'CUHK-IAMhaggler2012',
        'BramAgent2-AgentLG',
        'TheNegotiatorReloaded-CUHK',
        'TheNegotiatorReloaded-MetaAgent',
        'BramAgent2-OMACagent',
        'BramAgent2-AgentMR',
        'TheNegotiatorReloaded',
        'BramAgent2-TheNegotiatorReloaded',
        'MetaAgent',
        'OMACagent-AgentLG',
        'OMACagent-TheNegotiatorReloaded',
        'AgentLG',
        'IAMhaggler2012-TheNegotiatorReloaded',
        'AgentLG-TheNegotiatorReloaded',
        'TheNegotiatorReloaded-AgentLG',
        'BramAgent2-CUHK',
        'CUHK',
        'CUHK-AgentLG',
        'IAMhaggler2012-AgentLG',
        'BramAgent2',
        'MetaAgent-OMACagent',
        'CUHK-AgentMR',
        'MetaAgent-TheNegotiatorReloaded',
        'AgentMR-CUHK',
        'IAMhaggler2012-AgentMR',
        'OMACagent-AgentMR',
        'AgentMR-IAMhaggler2012',
        'TheNegotiatorReloaded-IAMhaggler2012',
        'AgentMR',
        'AgentLG-IAMhaggler2012',
        'AgentMR-MetaAgent',
        'OMACagent-BramAgent2',
        'MetaAgent-IAMhaggler2012',
        'AgentLG-OMACagent',
        'IAMhaggler2012-CUHK',
        'OMACagent-MetaAgent',
        'IAMhaggler2012',
        'TheNegotiatorReloaded-BramAgent2',
        'AgentLG-BramAgent2',
        'BramAgent2-IAMhaggler2012',
        'CUHK-MetaAgen